# A note about opening notebooks in shared workspaces <a class="tocSkip">

Master copies of notebooks should not be run or edited unless you intend to improve the code. As a general rule, it is good to be cautious when editing a notebook in a shared workspace, because you don't want to overwrite the work of your collaborators. Best practices is to test in a cloned workspace with an easily identifiable name.

# 1000 Genomes via bigrquery and dplyr

This notebook demonstrates two ways to use BigQuery with R
1. Using SQL syntax
2. Using only R code to extract the data of interest from BigQuery

# R-environment Setup

First, be sure to run the general notebook **`R environment setup`** in this workspace. 

Then run the cells to add additional needed libraries, set the project id, and authorize the BigQuery client. 

In [ ]:
# Load additional R libraries needed for this notebook into memory
library(bigrquery)
library(dplyr)
library(skimr)
library(ggplot2)

In [ ]:
# Set the project id of the clould project to bill for Queries to BigQuery
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

In [ ]:
# Authorize bigrquery client
bigrquery::set_service_token(Ronaldo::getServiceAccountKey())

# Retrieve filtered data
Let’s retrieve a subset of fields and samples in metadata for [1000 Genomes](http://www.internationalgenome.org/data "1000 Genomes").

We will do this in two different ways. Both return the same results.

1. Using SQL and bigrquery
2. Using dplyr and dbplyr

## Option 1: Retrieve filtered data using bigrquery

To learn more about SQL syntax see the [BigQuery standard SQL reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/).

In [ ]:
# Run a query with standard SQL 
phase1_samples_tbl <- bigrquery::bq_project_query(
    BILLING_PROJECT_ID,
    query = '
SELECT
  Sample,
  Gender,
  Relationship,
  Population,
  Population_Description,
  Super_Population,
  Super_Population_Description,
  Total_Exome_Sequence,
  Main_Project_E_Platform,
  Main_Project_E_Centers
FROM
  `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
WHERE
  -- Only include information for samples in phase 1.
  In_Phase1_Integrated_Variant_Set = TRUE
  -- Only include information for samples with these relationships.
  AND Relationship IN ("", "unrel", "child", "father", "mother",
  "mat grandmother", "pat grandmother",
  "mat grandfather", "pat grandfather")
')

phase1_samples <- bigrquery::bq_table_download(phase1_samples_tbl)

In [ ]:
dim(phase1_samples)

In [ ]:
print(skim(phase1_samples))

In [ ]:
ggplot(data = phase1_samples,
       aes(x=Relationship, fill=Relationship)) +
  geom_bar() +
  theme(axis.text.x = element_text(angle = 50, hjust = 1, vjust = 1))

## Option 2: Retrieve filtered data using dplyr
To learn more about dplyr see [R for Data Science Chapter 5 Data transformation](http://r4ds.had.co.nz/transform.html "Chapter 5").

In [ ]:
# Create a "connection" to a public BigQuery dataset.
dbcon <- bigrquery::src_bigquery(project = 'bigquery-public-data',
                                 dataset = 'human_genome_variants',
                                 billing = BILLING_PROJECT_ID)

# Create a 'virtual dataframe' backed by a BigQuery table.
sample_info <- dplyr::tbl(dbcon, '1000_genomes_sample_info')

In [ ]:
phase1_only <- sample_info %>% filter(
    In_Phase1_Integrated_Variant_Set == TRUE,
    Relationship %in% c(
      '', 'unrel', 'child', 'father', 'mother',
      'mat grandmother', 'pat grandmother',
      'mat grandfather', 'pat grandfather')
  )
# Nothing is computed on BigQuery yet.

In [ ]:
sample_fields <- phase1_only %>% select(
  Sample,
  Gender,
  Relationship,
  Population,
  Population_Description,
  Super_Population,
  Super_Population_Description,
  Total_Exome_Sequence,
  Main_Project_E_Platform,
  Main_Project_E_Centers
  )
# Still nothing computed on BigQuery.

In [ ]:
# Optional: take a look at the SQL.
dplyr::show_query(sample_fields)

In [ ]:
# Optional: See how much data this will return.
sample_fields %>% summarize(cnt = n()) %>% collect()

In [ ]:
# Execute the query and return all results into an in-memory table in R.
phase1_samples <- sample_fields %>% collect()

In [ ]:
dim(phase1_samples)

In [ ]:
ggplot(data = phase1_samples,
       aes(x=Relationship, fill=Relationship)) +
  geom_bar() +
  theme(axis.text.x = element_text(angle = 50, hjust = 1, vjust = 1))

# Provenance

In [ ]:
devtools::session_info()

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.